## Hiperparâmetros com Bayesian Optimization

O ajuste de hiperpârametros em um modelo de **Machine Learning** é extremamente importante, já que pode elevar substancialmente o desempenho do modelo em aderência aos dados.
É muito comum ver em tutoriais de Machine Learning o uso do **Grid Search** para ajustar os hiperparâmetros, entranto há um problema de custo de processamento alto já que o Grid Search irá executar vários modelos e testar todo o intervalo de hiperparâmetros definido pelo usuário.
Em contra partida o **Bayesian Optimization** é um método de ajuste de hiperparâmetros que utiliza a tecnologia de **Bayesian Inference** para encontrar o melhor valor de hiperparâmetros para o modelo.


#### Estudo
Usaremos um estudo já abordado em um tutorial aqui no blog: [Trading Machine - Decision Tree](https://jhonattanln.github.io/2022/05/12/Decision_Tree.html)
Onde basicamente o objetivo é classificar se o retorno do ativo será positivou ou negativo ao longo dos dias.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
